##### Importing the libraries and sets up the chatbot with the model and its configuration.

In [49]:
import gradio as gr
from langchain_groq import ChatGroq

# Here i am initialing the chatbot a model and a temperature
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.5,
    groq_api_key='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'
)


##### Defining preset responses for greetings and general questions to the chatbot.

In [51]:
# Here this code will predefine responses for greeting and general questions
greeting_responses = {
    "hello": "Hello! How can I help you today?",
    "hi": "Hi there! What would you like to talk about?",
    "hey": "Hey! I'm here to support you.",
    "good morning": "Good morning! How are you feeling today?",
    "good afternoon": "Good afternoon! How can I assist you?",
    "good evening": "Good evening! How's your day been?",
}

bot_related_responses = {
    "who are you": "I’m a mental health support chatbot. I'm here to provide empathetic responses and assist with questions related to the mind and emotions.",
    "what can you do": "I can help answer questions about mental health, thoughts, feelings, and other mind-related topics. Feel free to ask anything!",
    "tell me about yourself": "I’m designed to offer emotional support and help with mind-related topics, but I’m not a substitute for professional help. How can I assist you today?"
}


##### Function to determines is user's input is related to the mind or not.

In [53]:
# Here this code will check user input, that if it is related to mind related questions or not.
def is_mind_related_question(prompt):
    """
    Checks if the user's prompt is related to the mind, including various mental processes
    like thinking, visualizing, imagining, and others.
    """
    mind_related_keywords = [
        "mind", "brain", "thoughts", "thinking", "feelings", "emotions", "mental health", 
        "anxiety", "depression", "stress", "therapy", "counseling", "self-care", 
        "motivation", "focus", "concentration", "memory", "learning", "intelligence",
        "daydreaming", "mood", "meditation", "mindfulness", "behavior", "imagine",
        "visualize", "visualizing", "recall", "dream", "dreaming", "fantasize", "fantasy",
        "ruminate", "overthink", "self-talk", "inner dialogue", "mental imagery", 
        "cognition", "cognitive", "plan", "planning", "creative thinking", "subconscious"
    ]
    
    action_keywords = [
        "stop", "avoid", "control", "manage", "handle", "clear", "focus on", "concentrate on", 
        "remove", "stop thinking", "deal with", "visualize", "imagine", "plan", "process"
    ]

    # Converting our prompt to the lowercase letters
    prompt_lower = prompt.lower()

    # Cheking for mind keywords
    for keyword in mind_related_keywords:
        if keyword in prompt_lower:
            return True

    # Returning true if any of these case becomes true
    for action in action_keywords:
        if action in prompt_lower and ("thinking" in prompt_lower or "mind" in prompt_lower or "imagine" in prompt_lower):
            return True

    return False


##### Function to checks is the user's input is a greeting or a question to the chatbot and returning an appropriate predefined response.

In [55]:
def handle_greetings_or_bot_questions(prompt):
    """
    Handles simple greetings and general queries about the bot.
    """
    prompt_lower = prompt.lower()
    
    # This will check that is user prompt of greeting or not
    for greeting in greeting_responses:
        if greeting in prompt_lower:
            return greeting_responses[greeting]
    
    # This will check that is user prompt of general question to bot ot not.
    for query in bot_related_responses:
        if query in prompt_lower:
            return bot_related_responses[query]
    
    return None


##### The main logic for handling user query, checking for predefined responce, or generating chatbot replies based on the full conversation.

In [57]:
# This function will handle conversation history
def chatbot_response(user_prompt, chat_history):
    """
    Handles the user's input and appends to the conversation history.
    The chatbot generates responses based on the entire conversation history.
    """
    
    # Checking for greeitng or a general question of bot
    predefined_response = handle_greetings_or_bot_questions(user_prompt)
    
    if predefined_response:
        chat_history.append(("Bot", predefined_response))
        return chat_history

    # Appending the user quesry to the chat history for saving conversation
    chat_history.append(("User", user_prompt))
    
    # Checking if the user's input is related to the mind question or not
    if not is_mind_related_question(user_prompt):
        chat_history.append(("Bot", "Please ask only about questions related to the mind (e.g., thoughts, feelings, emotions, imagination)."))
        return chat_history
    
    # Here i prepare conversation and append to each previous one
    conversation = "\n".join([f"{speaker}: {message}" for speaker, message in chat_history])

    # Guiding propmp to help model resppond to it
    mind_related_prompt = f"I’m here to listen and offer support. Respond with empathy and understanding. Here is the conversation so far:\n{conversation}\n{user_prompt}"

    try:
        # Taking model response
        res = llm.invoke(mind_related_prompt)
        # Appending model response to chat
        chat_history.append(("Bot", res.content))
    except Exception as e:
        chat_history.append(("Bot", "Sorry, there was an issue processing your request. Please try again."))
    
    return chat_history


##### Creating the Gradio UI with options to submit, clear or just clear the input field.

In [65]:
# Creating gradio UI here
def launch_gradio_interface():
    # Creating textboxes for user input and output
    with gr.Blocks() as demo:
        # Adding a big title for the chatbot
        gr.Markdown("# 🧠 Mental Health Support Chatbot")

        # Using gradio's state to implement history
        chat_history = gr.State([])

        # Creating textbox and output box
        with gr.Row():
            user_input = gr.Textbox(label="Ask your mind-related question")
            chat_output = gr.Chatbot(label="Mental Health Support Chatbot")

        # Button for submitting query
        submit_button = gr.Button("Submit")

        # Updating chat history every time with user input
        def update_chat(user_prompt, chat_history):
            chat_history = chatbot_response(user_prompt, chat_history)
            return chat_history, chat_history, ""

        # Creating button to clear only the user prompt
        def clear_prompt():
            return ""

        # Creating button to clear both the user input and history
        def clear_chat():
            return [], [], ""

        # Creating a button to clear the user input only from UI
        clear_prompt_button = gr.Button("Clear Prompt")

        # Creating button to clear both the user input and the history
        clear_chat_button = gr.Button("Clear Chat")

        # Linking the user's input and the chat history's update
        submit_button.click(update_chat, inputs=[user_input, chat_history], outputs=[chat_output, chat_history, user_input])

        # Linking the clear buttons to their functions
        clear_prompt_button.click(clear_prompt, inputs=[], outputs=[user_input])
        clear_chat_button.click(clear_chat, inputs=[], outputs=[chat_output, chat_history, user_input])

    # Launch the Gradio UI
    demo.launch()


##### Final running the code

In [67]:
# Launching gradio interdace
launch_gradio_interface()

Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.
